<a href="https://colab.research.google.com/github/worldart/worldart_jupyternotebook/blob/main/BS%20and%20MC%20methods%20for%20Option%20Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import scipy.stats as ss


In [ ]:
#Price European call and put options using a Black-Scholes solution:

# Given parameters
S0 = 100
r = 0.05
sigma = 0.2
T = 0.25
K = 100

def black_scholes(S, K, r, sigma, T):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    call_price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    put_price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

    return call_price, put_price


# Calculate call and put option prices
call_price, put_price = black_scholes(S0, K, r, sigma, T)

print(f"European Call Option Price: {call_price:.2f}")
print(f"European Put Option Price: {put_price:.2f}")


European Call Option Price: 4.61
European Put Option Price: 3.37


In [ ]:
# Analyze the Greeks for input in Black-Scholes formula CALL Option:

# Given parameters
S = 100
r = 0.05
sigma = 0.2
T = 0.25
K = 100
vol = 0.20
option_type = "C"  # for the put insert 'P'

# dividend yield assumed to be 0

# Compute d1 and d2
d1 = (np.log(S / K) + (r + 0.5 * vol**2) * T) / (vol * np.sqrt(T))
d2 = d1 - vol * np.sqrt(T)

if option_type in ["C", "P"]:
    if option_type in ["C"]:
        Opt_Price = S * ss.norm.cdf(d1) - K * np.exp(-r * T) * ss.norm.cdf(d2)
        Delta = ss.norm.cdf(d1)
        Gamma = ss.norm.pdf(d1) / (S * vol * np.sqrt(T))
        Vega = S * ss.norm.pdf(d1) * np.sqrt(T)
        Theta = -(S * ss.norm.pdf(d1) * vol) / (2 * np.sqrt(T)) - r * K * np.exp(
            -r * T
        ) * ss.norm.cdf(d2)
        Rho = K * T * np.exp(-r * T) * ss.norm.cdf(d2)
    else:
        Opt_Price = K * np.exp(-r * T) * ss.norm.cdf(-d2) - S * ss.norm.cdf(-d1)
        Delta = -ss.norm.cdf(-d1)
        Gamma = ss.norm.pdf(d1) / (S * vol * np.sqrt(T))
        Vega = S * ss.norm.pdf(d1) * np.sqrt(T)
        Theta = -(S * ss.norm.pdf(d1) * vol) / (2 * np.sqrt(T)) + r * K * np.exp(
            -r * T
        ) * ss.norm.cdf(-d2)
        Rho = -K * T * np.exp(-r * T) * ss.norm.cdf(-d2)
else:
    Opt_Price = "Error: option type incorrect. Choose P for a put option or C for a call option."

print(f"Delta = {Delta:.2f}")
print(f"Gamma = {Gamma:.2f}")
print(f"Vega = {Vega:.2f}")
print(f"Theta = {Theta:.2f}")
print(f"Rho = {Rho:.2f}")

Delta = 0.57
Gamma = 0.04
Vega = 19.64
Theta = -10.47
Rho = 13.08


In [ ]:
# Analyze the Greeks for input in Black-Scholes formula PUT Option:

# Given parameters
S = 100
r = 0.05
sigma = 0.2
T = 0.25
K = 100
vol = 0.20
option_type = "P"

# dividend yield assumed to be 0

# Compute d1 and d2
d1 = (np.log(S / K) + (r + 0.5 * vol**2) * T) / (vol * np.sqrt(T))
d2 = d1 - vol * np.sqrt(T)

if option_type in ["C", "P"]:
    if option_type in ["C"]:
        Opt_Price = S * ss.norm.cdf(d1) - K * np.exp(-r * T) * ss.norm.cdf(d2)
        Delta = ss.norm.cdf(d1)
        Gamma = ss.norm.pdf(d1) / (S * vol * np.sqrt(T))
        Vega = S * ss.norm.pdf(d1) * np.sqrt(T)
        Theta = -(S * ss.norm.pdf(d1) * vol) / (2 * np.sqrt(T)) - r * K * np.exp(
            -r * T
        ) * ss.norm.cdf(d2)
        Rho = K * T * np.exp(-r * T) * ss.norm.cdf(d2)
    else:
        Opt_Price = K * np.exp(-r * T) * ss.norm.cdf(-d2) - S * ss.norm.cdf(-d1)
        Delta = -ss.norm.cdf(-d1)
        Gamma = ss.norm.pdf(d1) / (S * vol * np.sqrt(T))
        Vega = S * ss.norm.pdf(d1) * np.sqrt(T)
        Theta = -(S * ss.norm.pdf(d1) * vol) / (2 * np.sqrt(T)) + r * K * np.exp(
            -r * T
        ) * ss.norm.cdf(-d2)
        Rho = -K * T * np.exp(-r * T) * ss.norm.cdf(-d2)
else:
    Opt_Price = "Error: option type incorrect. Choose P for a put option or C for a call option."

print(f"Delta = {Delta:.2f}")
print(f"Gamma = {Gamma:.2f}")
print(f"Vega = {Vega:.2f}")
print(f"Theta = {Theta:.2f}")
print(f"Rho = {Rho:.2f}")

Delta = -0.43
Gamma = 0.04
Vega = 19.64
Theta = -5.54
Rho = -11.61


In [ ]:
#Price European call and put options using Monte-Carlo methods under a general GBM equation with daily time-steps in the simulations:

# Given parameters
S0 = 100
K = 100
r = 0.05
sigma = 0.2
T = 0.25
num_simulations = 100000
time_steps = 90


def monte_carlo_option_pricing(S0, K, r, sigma, T, num_simulations, time_steps):
    dt = T / time_steps
    Z = np.random.randn(num_simulations, time_steps)

    # Step 1: Simulate stock price paths according to SDE dynamics.
    S = np.zeros((num_simulations, time_steps + 1))
    S[:, 0] = S0
    for i in range(1, time_steps + 1):
        S[:, i] = S[:, i-1] * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z[:, i-1])

    # Step 2: Calculate option payoffs
    call_payoffs = np.maximum(S[:, -1] - K, 0)
    put_payoffs = np.maximum(K - S[:, -1], 0)

    # Step 3: Obtain the average expected payoff at maturity, T and will discount the average expected payoff to time t. As explained previously, we will use the T = T - t = 3/12 = 0.25 as parameter.
    call_price = np.mean(call_payoffs) * np.exp(-r*T)
    put_price = np.mean(put_payoffs) * np.exp(-r*T)

    return call_price, put_price


# Calculate call and put option prices
call_price, put_price = monte_carlo_option_pricing(S0, K, r, sigma, T, num_simulations, time_steps)

print(f"European Call Option Price: {call_price:.2f}")
print(f"European Put Option Price: {put_price:.2f}")

European Call Option Price: 4.63
European Put Option Price: 3.39


In [ ]:
# Analyze the Greeks for a European call option using Monte Carlo methods under a general Geometric Brownian Motion (GBM) with daily time steps

# Given parameters
S = 100
r = 0.05
sigma = 0.2
T = 0.25
K = 100
vol = 0.20
option_type = "C"  # "P" for put, "C" for call

# Simulation parameters
n_simulations = 100000  # Number of simulations
n_steps = int(252 * T)  # Number of daily time steps (252 trading days per year)
dt = T / n_steps  # Time step

# Monte Carlo simulation for asset paths
np.random.seed(42)  # Set seed for reproducibility
Z = np.random.standard_normal((n_simulations, n_steps))
S_t = np.zeros((n_simulations, n_steps + 1))
S_t[:, 0] = S

for t in range(1, n_steps + 1):
    S_t[:, t] = S_t[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Calculate option payoff at maturity
if option_type == "P":
    payoffs = np.maximum(K - S_t[:, -1], 0)
elif option_type == "C":
    payoffs = np.maximum(S_t[:, -1] - K, 0)
else:
    raise ValueError("Error: option type incorrect. Choose 'P' for a put option or 'C' for a call option.")

# Discounted expected payoff
Opt_Price = np.exp(-r * T) * np.mean(payoffs)

# Delta approximation
S_up = S + 1e-4  # Slightly increase initial stock price
S_down = S - 1e-4  # Slightly decrease initial stock price

# Simulate paths for S_up
S_t_up = np.zeros((n_simulations, n_steps + 1))
S_t_up[:, 0] = S_up
for t in range(1, n_steps + 1):
    S_t_up[:, t] = S_t_up[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Simulate paths for S_down
S_t_down = np.zeros((n_simulations, n_steps + 1))
S_t_down[:, 0] = S_down
for t in range(1, n_steps + 1):
    S_t_down[:, t] = S_t_down[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Calculate payoffs for S_up and S_down
if option_type == "P":
    payoffs_up = np.maximum(K - S_t_up[:, -1], 0)
    payoffs_down = np.maximum(K - S_t_down[:, -1], 0)
elif option_type == "C":
    payoffs_up = np.maximum(S_t_up[:, -1] - K, 0)
    payoffs_down = np.maximum(S_t_down[:, -1] - K, 0)

Opt_Price_up = np.exp(-r * T) * np.mean(payoffs_up)
Opt_Price_down = np.exp(-r * T) * np.mean(payoffs_down)

Delta = (Opt_Price_up - Opt_Price_down) / (2 * 1e-4)

# Gamma approximation
Gamma = (Opt_Price_up - 2 * Opt_Price + Opt_Price_down) / (1e-4)**2

# Vega approximation
vol_up = vol + 1e-4
vol_down = vol - 1e-4

# Simulate paths for vol_up
Z = np.random.standard_normal((n_simulations, n_steps))  # Re-generate noise
S_t_vol_up = np.zeros((n_simulations, n_steps + 1))
S_t_vol_up[:, 0] = S
for t in range(1, n_steps + 1):
    S_t_vol_up[:, t] = S_t_vol_up[:, t-1] * np.exp((r - 0.5 * vol_up**2) * dt + vol_up * np.sqrt(dt) * Z[:, t-1])

# Simulate paths for vol_down
S_t_vol_down = np.zeros((n_simulations, n_steps + 1))
S_t_vol_down[:, 0] = S
for t in range(1, n_steps + 1):
    S_t_vol_down[:, t] = S_t_vol_down[:, t-1] * np.exp((r - 0.5 * vol_down**2) * dt + vol_down * np.sqrt(dt) * Z[:, t-1])

# Calculate payoffs for vol_up and vol_down
payoffs_vol_up = np.maximum(K - S_t_vol_up[:, -1], 0)
payoffs_vol_down = np.maximum(K - S_t_vol_down[:, -1], 0)

Opt_Price_vol_up = np.exp(-r * T) * np.mean(payoffs_vol_up)
Opt_Price_vol_down = np.exp(-r * T) * np.mean(payoffs_vol_down)

Vega = (Opt_Price_vol_up - Opt_Price_vol_down) / (2 * 1e-4)

# Theta approximation
T_down = T - 1 / 252  # Reduce maturity by one day

# Simulate paths for T_down
S_t_T_down = np.zeros((n_simulations, n_steps + 1))
S_t_T_down[:, 0] = S
for t in range(1, n_steps):
    S_t_T_down[:, t] = S_t_T_down[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

payoffs_T_down = np.maximum(K - S_t_T_down[:, -1], 0)
Opt_Price_T_down = np.exp(-r * T_down) * np.mean(payoffs_T_down)

Theta = (Opt_Price_T_down - Opt_Price) / (1 / 252)

# Rho approximation
r_up = r + 1e-4
r_down = r - 1e-4

# Simulate paths for r_up and r_down
Opt_Price_r_up = np.exp(-r_up * T) * np.mean(payoffs)
Opt_Price_r_down = np.exp(-r_down * T) * np.mean(payoffs)

Rho = (Opt_Price_r_up - Opt_Price_r_down) / (2 * 1e-4)

print(f"Delta = {Delta:.2f}")
print(f"Gamma = {Gamma:.2f}")
print(f"Vega = {Vega:.2f}")
print(f"Theta = {Theta:.2f}")
print(f"Rho = {Rho:.2f}")


Delta = 0.57
Gamma = 0.09
Vega = 19.62
Theta = 23736.54
Rho = -1.15


In [ ]:
# Analyze the Greeks for a European put option using Monte Carlo methods under a general Geometric Brownian Motion (GBM) with daily time steps

# Given parameters
S = 100
r = 0.05
sigma = 0.2
T = 0.25
K = 100
vol = 0.20
option_type = "P"  # "P" for put, "C" for call

# Simulation parameters
n_simulations = 100000  # Number of simulations
n_steps = int(252 * T)  # Number of daily time steps (252 trading days per year)
dt = T / n_steps  # Time step

# Monte Carlo simulation for asset paths
np.random.seed(42)  # Set seed for reproducibility
Z = np.random.standard_normal((n_simulations, n_steps))
S_t = np.zeros((n_simulations, n_steps + 1))
S_t[:, 0] = S

for t in range(1, n_steps + 1):
    S_t[:, t] = S_t[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Calculate option payoff at maturity
if option_type == "P":
    payoffs = np.maximum(K - S_t[:, -1], 0)
elif option_type == "C":
    payoffs = np.maximum(S_t[:, -1] - K, 0)
else:
    raise ValueError("Error: option type incorrect. Choose 'P' for a put option or 'C' for a call option.")

# Discounted expected payoff
Opt_Price = np.exp(-r * T) * np.mean(payoffs)

# Delta approximation
S_up = S + 1e-4  # Slightly increase initial stock price
S_down = S - 1e-4  # Slightly decrease initial stock price

# Simulate paths for S_up
S_t_up = np.zeros((n_simulations, n_steps + 1))
S_t_up[:, 0] = S_up
for t in range(1, n_steps + 1):
    S_t_up[:, t] = S_t_up[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Simulate paths for S_down
S_t_down = np.zeros((n_simulations, n_steps + 1))
S_t_down[:, 0] = S_down
for t in range(1, n_steps + 1):
    S_t_down[:, t] = S_t_down[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Calculate payoffs for S_up and S_down
if option_type == "P":
    payoffs_up = np.maximum(K - S_t_up[:, -1], 0)
    payoffs_down = np.maximum(K - S_t_down[:, -1], 0)
elif option_type == "C":
    payoffs_up = np.maximum(S_t_up[:, -1] - K, 0)
    payoffs_down = np.maximum(S_t_down[:, -1] - K, 0)

Opt_Price_up = np.exp(-r * T) * np.mean(payoffs_up)
Opt_Price_down = np.exp(-r * T) * np.mean(payoffs_down)

Delta = (Opt_Price_up - Opt_Price_down) / (2 * 1e-4)

# Gamma approximation
Gamma = (Opt_Price_up - 2 * Opt_Price + Opt_Price_down) / (1e-4)**2

# Vega approximation
vol_up = vol + 1e-4
vol_down = vol - 1e-4

# Simulate paths for vol_up
Z = np.random.standard_normal((n_simulations, n_steps))  # Re-generate noise
S_t_vol_up = np.zeros((n_simulations, n_steps + 1))
S_t_vol_up[:, 0] = S
for t in range(1, n_steps + 1):
    S_t_vol_up[:, t] = S_t_vol_up[:, t-1] * np.exp((r - 0.5 * vol_up**2) * dt + vol_up * np.sqrt(dt) * Z[:, t-1])

# Simulate paths for vol_down
S_t_vol_down = np.zeros((n_simulations, n_steps + 1))
S_t_vol_down[:, 0] = S
for t in range(1, n_steps + 1):
    S_t_vol_down[:, t] = S_t_vol_down[:, t-1] * np.exp((r - 0.5 * vol_down**2) * dt + vol_down * np.sqrt(dt) * Z[:, t-1])

# Calculate payoffs for vol_up and vol_down
payoffs_vol_up = np.maximum(K - S_t_vol_up[:, -1], 0)
payoffs_vol_down = np.maximum(K - S_t_vol_down[:, -1], 0)

Opt_Price_vol_up = np.exp(-r * T) * np.mean(payoffs_vol_up)
Opt_Price_vol_down = np.exp(-r * T) * np.mean(payoffs_vol_down)

Vega = (Opt_Price_vol_up - Opt_Price_vol_down) / (2 * 1e-4)

# Theta approximation
T_down = T - 1 / 252  # Reduce maturity by one day

# Simulate paths for T_down
S_t_T_down = np.zeros((n_simulations, n_steps + 1))
S_t_T_down[:, 0] = S
for t in range(1, n_steps):
    S_t_T_down[:, t] = S_t_T_down[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

payoffs_T_down = np.maximum(K - S_t_T_down[:, -1], 0)
Opt_Price_T_down = np.exp(-r * T_down) * np.mean(payoffs_T_down)

Theta = (Opt_Price_T_down - Opt_Price) / (1 / 252)

# Rho approximation
r_up = r + 1e-4
r_down = r - 1e-4

# Simulate paths for r_up and r_down
Opt_Price_r_up = np.exp(-r_up * T) * np.mean(payoffs)
Opt_Price_r_down = np.exp(-r_down * T) * np.mean(payoffs)

Rho = (Opt_Price_r_up - Opt_Price_r_down) / (2 * 1e-4)

print(f"Delta = {Delta:.2f}")
print(f"Gamma = {Gamma:.2f}")
print(f"Vega = {Vega:.2f}")
print(f"Theta = {Theta:.2f}")
print(f"Rho = {Rho:.2f}")


Delta = -0.43
Gamma = 0.09
Vega = 19.62
Theta = 24043.99
Rho = -0.84


In [ ]:
# Use the Monte-Carlo methods with regular GBM process and daily simulations on an American Call option

# Given parameters
S0 = 100
r = 0.05
sigma = 0.2
T = 0.25
K = 100
n_simulations = 100000
n_steps = int(252 * T)
dt = T / n_steps

# Monte Carlo simulation for asset paths
np.random.seed(42)  # Set seed for reproducibility
Z = np.random.standard_normal((n_simulations, n_steps))
S_t = np.zeros((n_simulations, n_steps + 1))
S_t[:, 0] = S0

for t in range(1, n_steps + 1):
    S_t[:, t] = S_t[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Initialize cash flows at maturity
cash_flows = np.maximum(S_t[:, -1] - K, 0)


for t in range(n_steps-1, 0, -1):
    # Identify in-the-money paths
    in_the_money = np.where(S_t[:, t] > K)[0] # For Call, in-the-money if K < S_t
    if len(in_the_money) == 0:
        continue

    # Calculate the discounted cash flows for in-the-money paths
    discounted_cash_flows = np.exp(-r * dt) * cash_flows[in_the_money]

    X = S_t[in_the_money, t]
    Y = discounted_cash_flows
    poly_coeffs = np.polyfit(X, Y, 2)
    continuation_values = np.polyval(poly_coeffs, X)

    # Determine early exercise decision
    immediate_exercise_values = S_t[in_the_money, t] - K
    exercise_indices = np.where(immediate_exercise_values > continuation_values)[0]

    # Update cash flows
    cash_flows[in_the_money[exercise_indices]] = immediate_exercise_values[exercise_indices]

# Discount cash flows back to present value
option_price = np.mean(np.exp(-r * dt * np.arange(1, n_steps + 1))[:, None] * cash_flows) * np.exp(-r * t)

print(f"American Call Option Price = {option_price:.2f}")


American Call Option Price = 4.34


In [ ]:
# Use the Monte-Carlo methods with regular GBM process and daily simulations on an American Put option

# Given parameters
S0 = 100
r = 0.05
sigma = 0.2
T = 0.25
K = 100
n_simulations = 100000
n_steps = int(252 * T)
dt = T / n_steps

# Monte Carlo simulation for asset paths
np.random.seed(42)  # Set seed for reproducibility
Z = np.random.standard_normal((n_simulations, n_steps))
S_t = np.zeros((n_simulations, n_steps + 1))
S_t[:, 0] = S0

for t in range(1, n_steps + 1):
    S_t[:, t] = S_t[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Initialize cash flows at maturity
cash_flows = np.maximum(K - S_t[:, -1], 0)

for t in range(n_steps-1, 0, -1):
    # Identify in-the-money paths
    in_the_money = np.where(S_t[:, t] < K)[0]  # For Put, in-the-money if S_t < K
    if len(in_the_money) == 0:
        continue

    # Calculate the discounted cash flows for in-the-money paths
    discounted_cash_flows = np.exp(-r * dt) * cash_flows[in_the_money]

    X = S_t[in_the_money, t]
    Y = discounted_cash_flows
    poly_coeffs = np.polyfit(X, Y, 2)
    continuation_values = np.polyval(poly_coeffs, X)

    # Determine early exercise decision
    immediate_exercise_values = K - S_t[in_the_money, t]  # For Put, payoff is K - S_t
    exercise_indices = np.where(immediate_exercise_values > continuation_values)[0]

    # Update cash flows
    cash_flows[in_the_money[exercise_indices]] = immediate_exercise_values[exercise_indices]

# Discount cash flows back to present value
option_price = np.mean(np.exp(-r * dt * np.arange(1, n_steps + 1))[:, None] * cash_flows) * np.exp(-r * t)

print(f"American Put Option Price = {option_price:.2f}")


American Put Option Price = 3.31


In [ ]:
# Analyze the Greeks for an American call option using Monte Carlo methods under a general Geometric Brownian Motion (GBM) with daily time steps

# Given parameters
S = 100
r = 0.05
sigma = 0.2
T = 0.25
K = 100
vol = 0.20
option_type = "C"  # "C" for call, changed from "P"

# Simulation parameters
n_simulations = 100000  # Number of simulations
n_steps = int(252 * T)  # Number of daily time steps (252 trading days per year)
dt = T / n_steps  # Time step

# Monte Carlo simulation for asset paths
np.random.seed(42)  # Set seed for reproducibility
Z = np.random.standard_normal((n_simulations, n_steps))
S_t = np.zeros((n_simulations, n_steps + 1))
S_t[:, 0] = S

for t in range(1, n_steps + 1):
    S_t[:, t] = S_t[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Initialize the payoffs matrix for call option
payoffs = np.maximum(S_t - K, 0)

# Step backward through the tree to calculate the option price at each node
for t in range(n_steps-1, 0, -1):
    # Discounted expected payoff from continuation
    discounted_payoff = np.exp(-r * dt) * payoffs[:, t+1]

    # Identify paths where the option would be exercised
    in_the_money = np.where(S_t[:, t] > K)[0]

    if len(in_the_money) > 0:
        # Use linear regression to estimate the continuation value
        regression = np.polyfit(S_t[in_the_money, t], discounted_payoff[in_the_money], 2)
        continuation_value = np.polyval(regression, S_t[in_the_money, t])

        # Update the payoffs: max of exercise (intrinsic value) or continuation value
        payoffs[in_the_money, t] = np.maximum(payoffs[in_the_money, t], continuation_value)

    # Update the payoffs for early exercise
    payoffs[:, t] = np.where(payoffs[:, t] > discounted_payoff, payoffs[:, t], discounted_payoff)

# Option price is the average of the discounted payoff at the initial time step
Opt_Price = np.exp(-r * dt) * np.mean(payoffs[:, 1])

# Approximating the Greeks (as before but with early exercise considered)
# Delta approximation
S_up = S + 1e-4  # Slightly increase initial stock price
S_down = S - 1e-4  # Slightly decrease initial stock price

# Simulate paths for S_up
S_t_up = np.zeros((n_simulations, n_steps + 1))
S_t_up[:, 0] = S_up
for t in range(1, n_steps + 1):
    S_t_up[:, t] = S_t_up[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Simulate paths for S_down
S_t_down = np.zeros((n_simulations, n_steps + 1))
S_t_down[:, 0] = S_down
for t in range(1, n_steps + 1):
    S_t_down[:, t] = S_t_down[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Calculate payoffs for S_up and S_down
payoffs_up = np.maximum(S_t_up[:, -1] - K, 0)
payoffs_down = np.maximum(S_t_down[:, -1] - K, 0)

Opt_Price_up = np.exp(-r * T) * np.mean(payoffs_up)
Opt_Price_down = np.exp(-r * T) * np.mean(payoffs_down)

Delta = (Opt_Price_up - Opt_Price_down) / (2 * 1e-4)

# Gamma approximation
Gamma = (Opt_Price_up - 2 * Opt_Price + Opt_Price_down) / (1e-4)**2

# Vega approximation
vol_up = vol + 1e-4
vol_down = vol - 1e-4

# Simulate paths for vol_up
Z = np.random.standard_normal((n_simulations, n_steps))  # Re-generate noise
S_t_vol_up = np.zeros((n_simulations, n_steps + 1))
S_t_vol_up[:, 0] = S
for t in range(1, n_steps + 1):
    S_t_vol_up[:, t] = S_t_vol_up[:, t-1] * np.exp((r - 0.5 * vol_up**2) * dt + vol_up * np.sqrt(dt) * Z[:, t-1])

# Simulate paths for vol_down
S_t_vol_down = np.zeros((n_simulations, n_steps + 1))
S_t_vol_down[:, 0] = S
for t in range(1, n_steps + 1):
    S_t_vol_down[:, t] = S_t_vol_down[:, t-1] * np.exp((r - 0.5 * vol_down**2) * dt + vol_down * np.sqrt(dt) * Z[:, t-1])

# Calculate payoffs for vol_up and vol_down
payoffs_vol_up = np.maximum(S_t_vol_up[:, -1] - K, 0)
payoffs_vol_down = np.maximum(S_t_vol_down[:, -1] - K, 0)

Opt_Price_vol_up = np.exp(-r * T) * np.mean(payoffs_vol_up)
Opt_Price_vol_down = np.exp(-r * T) * np.mean(payoffs_vol_down)

Vega = (Opt_Price_vol_up - Opt_Price_vol_down) / (2 * 1e-4)

# Theta approximation
T_down = T - 1 / 252  # Reduce maturity by one day

# Simulate paths for T_down
S_t_T_down = np.zeros((n_simulations, n_steps + 1))
S_t_T_down[:, 0] = S
for t in range(1, n_steps):
    S_t_T_down[:, t] = S_t_T_down[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

payoffs_T_down = np.maximum(S_t_T_down[:, -1] - K, 0)
Opt_Price_T_down = np.exp(-r * T_down) * np.mean(payoffs_T_down)

Theta = (Opt_Price_T_down - Opt_Price) / (1 / 252)

# Rho approximation
r_up = r + 1e-4
r_down = r - 1e-4

# Simulate paths for r_up and r_down
Opt_Price_r_up = np.exp(-r_up * T) * np.mean(payoffs)
Opt_Price_r_down = np.exp(-r_down * T) * np.mean(payoffs)

Rho = (Opt_Price_r_up - Opt_Price_r_down) / (2 * 1e-4)

print(f"Delta = {Delta:.2f}")
print(f"Gamma = {Gamma:.2f}")
print(f"Vega = {Vega:.2f}")
print(f"Theta = {Theta:.2f}")
print(f"Rho = {Rho:.2f}")


Delta = 0.57
Gamma = -6859996763.73
Vega = 19.55
Theta = -9798.96
Rho = -4.71


In [ ]:
# Analyze the Greeks for an American put option using Monte Carlo methods under a general Geometric Brownian Motion (GBM) with daily time steps

# Given parameters
S = 100
r = 0.05
sigma = 0.2
T = 0.25
K = 100
vol = 0.20
option_type = "P"  # "P" for put, "C" for call

# Simulation parameters
n_simulations = 100000  # Number of simulations
n_steps = int(252 * T)  # Number of daily time steps (252 trading days per year)
dt = T / n_steps  # Time step

# Monte Carlo simulation for asset paths
np.random.seed(42)  # Set seed for reproducibility
Z = np.random.standard_normal((n_simulations, n_steps))
S_t = np.zeros((n_simulations, n_steps + 1))
S_t[:, 0] = S

for t in range(1, n_steps + 1):
    S_t[:, t] = S_t[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Initialize the payoffs matrix
payoffs = np.maximum(K - S_t, 0)

# Step backward through the tree to calculate the option price at each node
for t in range(n_steps-1, 0, -1):
    # Discounted expected payoff from continuation
    discounted_payoff = np.exp(-r * dt) * payoffs[:, t+1]

    # Identify paths where the option would be exercised
    in_the_money = np.where(payoffs[:, t] > 0)[0]

    if len(in_the_money) > 0:
        # Use linear regression to estimate the continuation value
        regression = np.polyfit(S_t[in_the_money, t], discounted_payoff[in_the_money], 2)
        continuation_value = np.polyval(regression, S_t[in_the_money, t])

        # Update the payoffs: max of exercise (intrinsic value) or continuation value
        payoffs[in_the_money, t] = np.maximum(payoffs[in_the_money, t], continuation_value)

    # Update the payoffs for early exercise
    payoffs[:, t] = np.where(payoffs[:, t] > discounted_payoff, payoffs[:, t], discounted_payoff)

# Option price is the average of the discounted payoff at the initial time step
Opt_Price = np.exp(-r * dt) * np.mean(payoffs[:, 1])

# Approximating the Greeks (as before but with early exercise considered)
# Delta approximation
S_up = S + 1e-4  # Slightly increase initial stock price
S_down = S - 1e-4  # Slightly decrease initial stock price

# Simulate paths for S_up
S_t_up = np.zeros((n_simulations, n_steps + 1))
S_t_up[:, 0] = S_up
for t in range(1, n_steps + 1):
    S_t_up[:, t] = S_t_up[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Simulate paths for S_down
S_t_down = np.zeros((n_simulations, n_steps + 1))
S_t_down[:, 0] = S_down
for t in range(1, n_steps + 1):
    S_t_down[:, t] = S_t_down[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

# Calculate payoffs for S_up and S_down
payoffs_up = np.maximum(K - S_t_up[:, -1], 0)
payoffs_down = np.maximum(K - S_t_down[:, -1], 0)

Opt_Price_up = np.exp(-r * T) * np.mean(payoffs_up)
Opt_Price_down = np.exp(-r * T) * np.mean(payoffs_down)

Delta = (Opt_Price_up - Opt_Price_down) / (2 * 1e-4)

# Gamma approximation
Gamma = (Opt_Price_up - 2 * Opt_Price + Opt_Price_down) / (1e-4)**2

# Vega approximation
vol_up = vol + 1e-4
vol_down = vol - 1e-4

# Simulate paths for vol_up
Z = np.random.standard_normal((n_simulations, n_steps))  # Re-generate noise
S_t_vol_up = np.zeros((n_simulations, n_steps + 1))
S_t_vol_up[:, 0] = S
for t in range(1, n_steps + 1):
    S_t_vol_up[:, t] = S_t_vol_up[:, t-1] * np.exp((r - 0.5 * vol_up**2) * dt + vol_up * np.sqrt(dt) * Z[:, t-1])

# Simulate paths for vol_down
S_t_vol_down = np.zeros((n_simulations, n_steps + 1))
S_t_vol_down[:, 0] = S
for t in range(1, n_steps + 1):
    S_t_vol_down[:, t] = S_t_vol_down[:, t-1] * np.exp((r - 0.5 * vol_down**2) * dt + vol_down * np.sqrt(dt) * Z[:, t-1])

# Calculate payoffs for vol_up and vol_down
payoffs_vol_up = np.maximum(K - S_t_vol_up[:, -1], 0)
payoffs_vol_down = np.maximum(K - S_t_vol_down[:, -1], 0)

Opt_Price_vol_up = np.exp(-r * T) * np.mean(payoffs_vol_up)
Opt_Price_vol_down = np.exp(-r * T) * np.mean(payoffs_vol_down)

Vega = (Opt_Price_vol_up - Opt_Price_vol_down) / (2 * 1e-4)

# Theta approximation
T_down = T - 1 / 252  # Reduce maturity by one day

# Simulate paths for T_down
S_t_T_down = np.zeros((n_simulations, n_steps + 1))
S_t_T_down[:, 0] = S
for t in range(1, n_steps):
    S_t_T_down[:, t] = S_t_T_down[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t-1])

payoffs_T_down = np.maximum(K - S_t_T_down[:, -1], 0)
Opt_Price_T_down = np.exp(-r * T_down) * np.mean(payoffs_T_down)

Theta = (Opt_Price_T_down - Opt_Price) / (1 / 252)

# Rho approximation
r_up = r + 1e-4
r_down = r - 1e-4

# Simulate paths for r_up and r_down
Opt_Price_r_up = np.exp(-r_up * T) * np.mean(payoffs)
Opt_Price_r_down = np.exp(-r_down * T) * np.mean(payoffs)

Rho = (Opt_Price_r_up - Opt_Price_r_down) / (2 * 1e-4)

print(f"Delta = {Delta:.2f}")
print(f"Gamma = {Gamma:.2f}")
print(f"Vega = {Vega:.2f}")
print(f"Theta = {Theta:.2f}")
print(f"Rho = {Rho:.2f}")


Delta = -0.43
Gamma = -4312212219.59
Vega = 19.62
Theta = 18610.61
Rho = -3.17


In [ ]:
# Chapter 3 Q 7: Price an European Call option with 110% moneyness and an European Put with 95% moneyness using a Black-Scholes model
# Given parameters
S0 = 100
r = 0.05
sigma = 0.2
T = 0.25

# Strike prices for call and put
K_C = 110
K_P = 95

# Calculating d1 and d2 for call option
d1_C = (np.log(S0/K_C) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
d2_C = d1_C - sigma * np.sqrt(T)

# Calculating d1 and d2 for put option
d1_P = (np.log(S0/K_P) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
d2_P = d1_P - sigma * np.sqrt(T)

# Calculating call and put prices using Black-Scholes
C = S0 * norm.cdf(d1_C) - K_C * np.exp(-r * T) * norm.cdf(d2_C)
P = K_P * np.exp(-r * T) * norm.cdf(-d2_P) - S0 * norm.cdf(-d1_P)

# Delta for call and put options
Delta_C = norm.cdf(d1_C)
Delta_P = norm.cdf(d1_P) - 1

# Portfolio Deltas
Delta_portfolio_B = Delta_C + Delta_P  # Long Call + Long Put
Delta_portfolio_C = Delta_C - Delta_P  # Long Call + Short Put

C, P, Delta_portfolio_B, Delta_portfolio_C

print(f"European Call Option Price (with 110% moneyness):C = {C:.2f}")
print(f"European Put Option Price (with 95% moneyness):P = {P:.2f}")

print(f"Delta for Call option = {Delta_C:.2f}")
print(f"Delta for Put option = {Delta_P:.2f}")

print(f"Delta for portfolio B = {Delta_portfolio_B:.2f}")
print(f"Delta for portfolio C = {Delta_portfolio_C:.2f}")


European Call Option Price (with 110% moneyness):C = 1.19
European Put Option Price (with 95% moneyness):P = 1.53
Delta for Call option = 0.22
Delta for Put option = -0.25
Delta for portfolio B = -0.03
Delta for portfolio C = 0.46


In [ ]:
#Use Monte-Carlo methods with daily time steps to price an Up-and-Out (UAO) barrier option.

# Given parameters
S0 = 120
r = 0.06
sigma = 0.30
T = 0.6667
K = S0
B = 141
n_simulations = 100000
n_steps = 252

# Time step size
dt = T / n_steps

# Pre-compute constants
drift = (r - 0.5 * sigma**2) * dt
diffusion = sigma * np.sqrt(dt)

# Monte Carlo simulation
np.random.seed(42)  # Set seed for reproducibility
payoffs = np.zeros(n_simulations)

for i in range(n_simulations):
    S_t = S0
    breach = False

    for t in range(n_steps):
        # Generate a random standard normal variable
        Z = np.random.standard_normal()

        # Update the asset price
        S_t *= np.exp(drift + diffusion * Z)

        # Check if the barrier is breached
        if S_t >= B:
            breach = True
            break

    # Payoff calculation (Call option)
    if not breach:
        payoffs[i] = max(S_t - K, 0)

# Calculate the option price as the discounted average of the payoffs
option_price = np.exp(-r * T) * np.mean(payoffs)

print(f"Up-and-Out Barrier Option Price using Monte Carlo: {option_price:.2f}")

Up-and-Out Barrier Option Price using Monte Carlo: 0.69


In [ ]:
# Use Monte-Carlo methods with daily time steps to price an an Up-and-In barrier (UAI) option.

# Given parameters
S0 = 120
K = 120
B = 141
r = 0.06
sigma = 0.30
T = 0.6667
N = 100000
steps = 252

# Calculate time step size
dt = T / steps

# Simulate N paths of the stock price
np.random.seed(42)  # For reproducibility
payoffs = np.zeros(N)

for i in range(N):
    # Generate a path of the stock price
    prices = np.zeros(steps)
    prices[0] = S0
    knocked_in = False

    for t in range(1, steps):
        # GBM model
        z = np.random.normal()
        prices[t] = prices[t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)

        # Check for barrier breach
        if prices[t] >= B:
            knocked_in = True

    # Calculate the payoff if the barrier is breached
    if knocked_in:
        payoffs[i] = max(0, prices[-1] - K)  # Call option payoff (max(0, S_T - K))

# Discount the payoff back to the present
price = np.exp(-r * T) * np.mean(payoffs)

# Output the result
print(f"Up-and-In Barrier Option Price using Monte Carlo: {price:.2f}")


Up-and-In Barrier Option Price using Monte Carlo: 13.12


In [ ]:
# Compute the price of the vanilla option with same characteristics in the scenario, but without any barrier.

# Given parameters
S0 = 120
K = 120
B = 141
r = 0.06
sigma = 0.30
T = 0.6667
N = 100000
steps = 252

# Calculate time step size
dt = T / steps

# Simulate N paths of the stock price
np.random.seed(42)  # For reproducibility
payoffs = np.zeros(N)

for i in range(N):
    # Generate a path of the stock price
    prices = np.zeros(steps)
    prices[0] = S0

    for t in range(1, steps):
        # GBM model
        z = np.random.normal()
        prices[t] = prices[t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)

    # Calculate the payoff of the vanilla call option
    payoffs[i] = max(0, prices[-1] - K)  # Call option payoff (max(0, S_T - K))

# Discount the payoff back to the present
vanilla_price = np.exp(-r * T) * np.mean(payoffs)

# Output the result
print(f"Vanilla Option Price using Monte Carlo: {vanilla_price:.2f}")


Vanilla Option Price using Monte Carlo: 13.80
